In [1]:
cd ..

D:\J.H.LEE\05. CODING\Python\Project\tdatlib


In [2]:
# For Library
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
from datetime import timedelta

# For Testing
import tdatlib as tdat
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important;}</style>"))

# LIB DEV

## Functionality

In [4]:
def normalize(time_series:pd.Series or np.array, a:float=0, b:float=1):
    return (b - a) * (time_series - time_series.min()) / (time_series.max() - time_series.min()) + a

In [5]:
def price_rel(time_series:pd.Series or pd.DataFrame, init:float=100):
    return init * ((time_series.dropna().pct_change() + 1).cumprod() - 1)

In [6]:
def price_hlc(ohlcv:pd.DataFrame) -> pd.Series:
    if not ('종가' in ohlcv.columns and '고가' in ohlcv.columns and '저가' in ohlcv.columns):
        raise KeyError
    return (1/3) * ohlcv.종가 + (1/3) * ohlcv.고가 + (1/3) * ohlcv.저가

In [12]:
def corrcoeff(l:pd.Series, r:pd.Series):
    join = pd.concat(objs=[l, r], axis=1).dropna()
    join = join[join.index.isin(l.index)]
    return join.corr(method='pearson', min_periods=1).iloc[0, 1]

# TESTER

## BASIC DATASET

### Handler

In [8]:
ecos = tdat.macro.ecos()
fred = tdat.macro.fred()
index = tdat.market.index()

period = 10
ecos.period = fred.period = index.period = period

ticker = '105560'
stock = tdat.stock.kr(ticker=ticker)
stock.period = period

### Indicator

In [18]:
cols = ['시가', '고가', '저가', '종가']
kospi = index.kospi
krbank = index.bank
exchange = ecos.원달러환율

ohlcv = stock.ohlcv
pd.concat(objs={stock.name:ohlcv[cols], '환율':exchange, '은행':krbank[cols]}, axis=1)

KB금융                           환율                          \
               시가     고가     저가     종가      시가      고가      저가      종가   
2012-10-22  38500  39000  38050  39000  1106.0  1106.5  1103.7  1104.2   
2012-10-23  38700  38850  37450  38100  1101.5  1103.4  1100.0  1103.1   
2012-10-24  37650  38200  37450  37750  1106.0  1106.0  1102.6  1103.6   
2012-10-25  37300  37600  36850  37300  1103.4  1103.5  1097.7  1098.2   
2012-10-26  37200  37950  37000  37000  1096.5  1098.3  1094.8  1097.0   
...           ...    ...    ...    ...     ...     ...     ...     ...   
2022-10-13  44700  45600  44500  44700  1424.5  1431.4  1423.2  1431.3   
2022-10-14  45550  47100  45550  46750  1431.5  1434.2  1425.0  1428.5   
2022-10-17  46000  46950  45800  45900  1440.9  1441.4  1432.4  1435.3   
2022-10-18  46500  47600  46350  47350  1430.0  1431.3  1418.7  1422.7   
2022-10-19  47700  48150  47400  47950     NaN     NaN     NaN     NaN   

                은행                          
                시가      고가      저가      종가  
2012-10-22  702.56  710.71  702.09  710.65  
2012-10-23  708.44  710.14  695.50  699.11  
2012-10-24  692.08  698.34  690.80  692.75  
2012-10-25  687.32  691.28  684.14  690.57  
2012-10-26  686.32  696.73  683.42  685.23  
...            ...     ...     ...     ...  
2022-10-13  553.55  559.55  550.02  551.17  
2022-10-14  561.64  574.50  560.62  570.08  
2022-10-17  561.19  567.98  560.58  565.43  
2022-10-18  576.46  579.59  570.26  578.07  
2022-10-19  581.04  589.41  581.04  583.32  

[2458 rows x 12 columns]

## TESTING DATASET

In [19]:
basic = corrcoef(krbank.종가, exchange.종가)
hlc = corrcoeff(price_hlc(krbank), price_hlc(exchange))
norm = corrcoeff(normalize(krbank.종가), normalize(exchange.종가))
rel = corrcoeff(price_rel(krbank.종가), price_rel(exchange.종가))

print(basic)
print(hlc)
print(norm)
print(rel)

KB금융
-0.5091720178091894
-0.5102482107499312
-0.5091720178091849
-0.5092586385849106


## VISUALIZE